# Data Processing 1

## 1. 온라인 행동데이터 내 검색 키워드(sech_kwd)를 활용한 상품 대분류(clac_nm1) 매핑

 * 1. 온라인 행동데이터 내 검색 키워드(1-10.sech_kwd)은 약 10만개의 단어로 이루어짐
 * 이 중에서 3번 이상 검색된 키워드(29900개)를 추출
 * 웹 스크래이핑을 활용하여 추출한 단어를 롯데홈쇼핑 메인 페이지에서 검색한 후, 나오는 결과 중
   '카테고리' 창에 나오는 키워드를 추출하여 테이블로 저장
 * 해당 키워드 중 2번째 키워드(검색어에 대한 가장 많은 수의 물품에 대한 분류명)와 유사한 상품 대분류명(4-2.clac_nm1)을 매핑하여 입력 데이터의
   컬럼으로 추가

 * 라이브러리 호출

In [90]:
import numpy as np
import pandas as pd
import os
from collections import Counter
import datetime
from random import *
import re
import requests
from tqdm import tqdm


import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib.font_manager import fontManager
import plotly as py
import plotly.express as px
from pprint import pprint

import lxml
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import seaborn as sns
import cufflinks as cf
%matplotlib notebook
cf.go_offline()


 * 폰트

In [91]:
for font in fontManager.ttflist:
    if 'Mal' in font.name:
        print(font.name)
plt.rc('font', family='Malgun Gothic')

Lohit Malayalam
Samyak Malayalam
RaghuMalayalam


 * 데이터 파일 불러오기
     * merge_raw.cav = 1. 온라인 행동 정보, 2. 거래정보, 3 사용자정보, 4.상품분류명을 모두 합친 데이터

In [92]:
# col_dtype = {'act_kwd':'str'}

In [134]:
merge_table = pd.read_csv('./merge_raw_1234.csv')
# merge_table = pd.read_csv('./merge_raw_1234.csv', dtype=col_dtype)
merge_table = merge_table.drop(['Unnamed: 0'], axis=1)

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (17,27) have mixed types.Specify dtype option on import or set low_memory=False.



In [135]:
merge_table.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,de_dt,de_tm,buy_am,buy_ct,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2
0,72429,1,2,2,A03,20190919,22:09,842130,NaN,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
1,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
2,72428,5,1,0,A01,20190918,06:37,12568,NaN,라인에디션블라우스,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
3,72428,4,1,0,A01,20190914,23:42,243555,NaN,펜디가방,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
4,72428,3,3,0,A01,20190914,21:50,839771,NaN,페레가모가방,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN


 * 3번 이상 검색된 단어 불러오기

In [136]:
sech_kwd = merge_table['sech_kwd'].value_counts()
sech_kwd = sech_kwd[sech_kwd >= 3]
sech_kwd

우유           8985
두부           5210
계란           5039
생수           4283
수박           2694
             ... 
생삼겹살            3
오뚜기 꼬리곰탕        3
wd40            3
더나무             3
에스티엘여성수영복       3
Name: sech_kwd, Length: 29900, dtype: int64

In [137]:
sech_kwd_list = list(sech_kwd.index)
sech_kwd_list

['우유',
 '두부',
 '계란',
 '생수',
 '수박',
 '라면',
 '오이',
 '콩나물',
 '쌀',
 '식빵',
 '어묵',
 '양파',
 '과자',
 '바나나',
 '당근',
 '상추',
 '삼겹살',
 '햇반',
 '요구르트',
 '복숭아',
 '김',
 '버섯',
 '떡볶이',
 '요플레',
 '원피스',
 '김치',
 '대파',
 '만두',
 '닭',
 '치즈',
 '아이스크림',
 '시금치',
 '컵라면',
 '햄',
 '애호박',
 '감자',
 '삼다수',
 '양배추',
 '깻잎',
 '신라면',
 '토마토',
 '짜파게티',
 '오징어',
 '빵',
 '요거트',
 '사과',
 '카레',
 '콜라',
 '맛살',
 '양상추',
 '단무지',
 '파프리카',
 '부추',
 '커피',
 '쿠킹박스',
 '서울우유',
 '고구마',
 '쌀10kg',
 '무',
 '돼지고기',
 '소고기',
 '사이다',
 '휴지',
 '포도',
 '마늘',
 '키친타올',
 '호박',
 '달걀',
 '비비고',
 '베이컨',
 '파',
 '두유',
 '샐러드',
 '핫도그',
 '물티슈',
 '소세지',
 '참치',
 '순두부',
 '당면',
 '마요네즈',
 '고추장',
 '계란30',
 '새우',
 '불고기',
 '스팸',
 '식용유',
 '물',
 '버터',
 '진라면',
 '닭가슴살',
 '유부초밥',
 '오뎅',
 '바나나우유',
 '배추',
 '치약',
 '식초',
 '훈제오리',
 '케찹',
 '생리대',
 '설탕',
 '냉면',
 '샴푸',
 '쌀20kg',
 '간장',
 '숙주',
 '화장지',
 '비엔나',
 '김밥',
 '주방세제',
 '자두',
 '김밥김',
 '돈까스',
 '피자',
 '쌈무',
 '쌈장',
 '카프리썬',
 '귤',
 '삼계탕',
 '메추리알',
 '한우',
 '과일',
 '옥수수',
 '음료수',
 '오리',
 '탄산수',
 '떡볶이떡',
 '비빔면',
 '참기름',
 '생수2l',
 '브로콜리',
 '미역',
 

In [138]:
len(sech_kwd_list)

29900

 * 3번 이상 검색된 단어만 따로 열 만들기

In [139]:
merge_table['act_kwd_over3'] = merge_table['act_kwd'][merge_table['act_kwd'].isin(sech_kwd_list)]
merge_table['act_kwd_over3']

0             NaN
1             NaN
2             NaN
3            펜디가방
4          페레가모가방
            ...  
3196357       NaN
3196358       NaN
3196359     과일바구니
3196360       NaN
3196361    과일선물세트
Name: act_kwd_over3, Length: 3196362, dtype: object

In [140]:
merge_table['act_kwd_over3'].nunique()

29900

* 키워드 리스트 불러오기

In [100]:
act_kwd_over3 = list(merge_table['act_kwd_over3'].unique())
act_kwd_over3.remove(np.nan)
act_kwd_over3

['펜디가방',
 '페레가모가방',
 '루이비통',
 '구찌',
 '속옷세트',
 '핫도그',
 '원피스',
 '아이패스파워',
 '계란',
 'enqn',
 '멜빵바지',
 '보쌈용',
 '삼겹',
 '보쌈',
 '수박',
 '앞다리',
 '찌개용',
 '비비고포기김치',
 '미숫가루',
 '햇반210g',
 '계란30',
 '종근당 락토핏',
 '비엔나',
 '팸퍼스',
 '호두',
 '닭갈비',
 '피쇼',
 '펩시',
 '골뱅이',
 '쌀',
 '맥스클리닉',
 '빠질라카노',
 '구찌 지갑',
 '남자구찌지갑',
 '프라다 지갑',
 '빈폴지갑',
 '프라다지갑',
 '남자지갑',
 '몽블랑지갑',
 '지갑',
 '베이퍼맥스',
 '떡',
 '비누',
 '천연비누',
 '밍크코트',
 '가죽잠바',
 '고추장',
 '최유라쇼',
 '닥스양산',
 '양산',
 '양산 uv차단',
 '키친타올',
 '서큘레이터',
 '파세코',
 '르젠',
 '리리코스',
 '파세코 써큘레이터',
 '고시히카리',
 '잘모이',
 '파타고니아후리스',
 '물걸레청소포',
 '써스데이아일랜드원피스',
 '만두피',
 '생만두피',
 '냉동만두피',
 '왕만두피',
 '닭발',
 '라인댄스화',
 '발톱네일아트',
 '발톱레일',
 '상품권',
 '기프트카드',
 '김',
 '양배추',
 '오일',
 '보테가베네타남자지갑',
 '온앤온',
 '코렐',
 '오덴세',
 '루카스나인라떼',
 '바지걸이',
 '스타벅스',
 '▣셔츠',
 '흄',
 '복합기',
 '로퍼',
 '소보제화',
 '고데기',
 '봉고데기',
 '뽕고데기',
 '손목지갑',
 '손목시계',
 '집게핀',
 '립 마에스트로',
 '헤지스키즈',
 '헤지스레이디스 원피스',
 '셔츠원피스',
 '여성 레이어드 티셔츠',
 '여성 롱 티셔츠',
 '꼬망스',
 '빈 원피스',
 '매니큐어',
 '샤넬 르베르니',
 '백산수',
 '흑설탕',
 '조거팬츠 여성',
 '선풍기커버',
 '흙침대',
 '프레쉬 립

In [ ]:
-------------------------------------------------------------------------------------------------------------------

 * 시범 삼아 1개 추출

In [13]:
# Selenium이 사용할 브라우저 선택 및 크롬 드라이버 실행
driver=webdriver.Chrome('./chromedriver')

In [78]:
# 웹 페이지를 읽어오기
driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')

# find_element_by_id : id 속성으로 요소 하나 추출

a_element1=driver.find_element_by_id('headerQuery') # headerQuery : 검색어 입력 창
a_element1.click()                                  # 해당 요소를 클릭
a_element1.send_keys('로가디스')                     # 키를 입력


a_element2=driver.find_element_by_id("btn_headerSearch") # btn_headerSearch : 검색 버튼
a_element2.click()

a_element3=driver.find_element_by_class_name("content_category")

In [79]:
a_element3.text

'패션의류/언더웨어\n남성정장/셔츠 (1,133)\n남성캐주얼 (822)\n진/유니섹스 (52)\n명품/쥬얼리\n시계/보석/쥬얼리 (221)\n스포츠/레저\n스포츠패션/슈즈 (37)\n영플라자\n진/유니섹스 (26)\n롯데백화점\n진/유니섹스 (12)\n남성정장/셔츠 (3)\n남성캐주얼 (1)'

In [80]:
a_element4=a_element3.text.split('\n')

In [81]:
a_element4

['패션의류/언더웨어',
 '남성정장/셔츠 (1,133)',
 '남성캐주얼 (822)',
 '진/유니섹스 (52)',
 '명품/쥬얼리',
 '시계/보석/쥬얼리 (221)',
 '스포츠/레저',
 '스포츠패션/슈즈 (37)',
 '영플라자',
 '진/유니섹스 (26)',
 '롯데백화점',
 '진/유니섹스 (12)',
 '남성정장/셔츠 (3)',
 '남성캐주얼 (1)']

In [82]:
test=[]
for i in range(len(a_element4)):
    text = re.sub(r'\([^)]*\)', repl='', string=a_element4[i])
    text=text.rstrip()
    test.append(text)
a_element5=pd.Series(test,name=1)

In [83]:
a_element5

0     패션의류/언더웨어
1       남성정장/셔츠
2         남성캐주얼
3        진/유니섹스
4        명품/쥬얼리
5     시계/보석/쥬얼리
6        스포츠/레저
7      스포츠패션/슈즈
8          영플라자
9        진/유니섹스
10        롯데백화점
11       진/유니섹스
12      남성정장/셔츠
13        남성캐주얼
Name: 1, dtype: object

In [ ]:
-------------------------------------------------------------------------------------------------------------------

In [49]:
act_kwd_cat=pd.Series(['시작'],name='start')
act_kwd_cat

driver=webdriver.Chrome('./chromedriver')  # Selenium이 사용할 브라우저 선택 및 크롬 드라이버 실행

# 1000개만 먼저
# 열 추가 방식
for i in tqdm(range(1000)):
    
#     driver.implicitly_wait(2)
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd_over3[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

# 파일 저장
act_kwd_cat.to_csv("sample_over3_1000.csv", mode='w')

100%|██████████| 1000/1000 [56:17<00:00,  3.38s/it] 


 * 1000 ~ 2000 번

In [59]:
act_kwd_cat=pd.Series(['시작'],name='start')
act_kwd_cat

driver=webdriver.Chrome('./chromedriver')  # Selenium이 사용할 브라우저 선택 및 크롬 드라이버 실행

# 1000개만 먼저
# 열 추가 방식
for i in tqdm(range(29000, 29900)):
    
#     driver.implicitly_wait(2)
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd_over3[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

# 파일 저장
act_kwd_cat.to_csv("sample_over3_29900.csv", mode='w')

100%|██████████| 900/900 [45:46<00:00,  3.05s/it]  


In [ ]:
-------------------------------------------------------------------------------------------------------------------

 * 롯데 홈쇼핑몰에서 추출한 카테고리 키워드 가져오기

In [101]:
cat_kwd_1 = pd.read_csv("./sample_over3_1000.csv")
cat_kwd_1 = cat_kwd_1.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_2 = pd.read_csv("./sample_over3_2000.csv")
cat_kwd_2 = cat_kwd_2.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_3 = pd.read_csv("./sample_over3_3000.csv")
cat_kwd_3 = cat_kwd_3.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_4 = pd.read_csv("./sample_over3_4000.csv")
cat_kwd_4 = cat_kwd_4.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_5 = pd.read_csv("./sample_over3_5000.csv")
cat_kwd_5 = cat_kwd_5.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_6 = pd.read_csv("./sample_over3_6000.csv")
cat_kwd_6 = cat_kwd_6.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_7 = pd.read_csv("./sample_over3_7000.csv")
cat_kwd_7 = cat_kwd_7.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_8 = pd.read_csv("./sample_over3_8000.csv")
cat_kwd_8 = cat_kwd_8.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_9 = pd.read_csv("./sample_over3_9000.csv")
cat_kwd_9 = cat_kwd_9.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_10 = pd.read_csv("./sample_over3_10000.csv")
cat_kwd_10 = cat_kwd_10.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_11 = pd.read_csv("./sample_over3_11000.csv")
cat_kwd_11 = cat_kwd_11.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_12 = pd.read_csv("./sample_over3_12000.csv")
cat_kwd_12 = cat_kwd_12.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_13 = pd.read_csv("./sample_over3_13000.csv")
cat_kwd_13 = cat_kwd_13.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_14 = pd.read_csv("./sample_over3_14000.csv")
cat_kwd_14 = cat_kwd_14.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_15 = pd.read_csv("./sample_over3_15000.csv")
cat_kwd_15 = cat_kwd_15.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_16 = pd.read_csv("./sample_over3_16000.csv")
cat_kwd_16 = cat_kwd_16.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_17 = pd.read_csv("./sample_over3_17000.csv")
cat_kwd_17 = cat_kwd_17.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_18 = pd.read_csv("./sample_over3_18000.csv")
cat_kwd_18 = cat_kwd_18.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_19 = pd.read_csv("./sample_over3_19000.csv")
cat_kwd_19 = cat_kwd_19.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_20 = pd.read_csv("./sample_over3_20000.csv")
cat_kwd_20 = cat_kwd_20.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_21 = pd.read_csv("./sample_over3_21000.csv")
cat_kwd_21 = cat_kwd_21.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_22 = pd.read_csv("./sample_over3_22000.csv")
cat_kwd_22 = cat_kwd_22.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_23 = pd.read_csv("./sample_over3_23000.csv")
cat_kwd_23 = cat_kwd_23.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_24 = pd.read_csv("./sample_over3_24000.csv")
cat_kwd_24 = cat_kwd_24.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_25 = pd.read_csv("./sample_over3_25000.csv")
cat_kwd_25 = cat_kwd_25.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_26 = pd.read_csv("./sample_over3_26000.csv")
cat_kwd_26 = cat_kwd_26.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_27 = pd.read_csv("./sample_over3_27000.csv")
cat_kwd_27 = cat_kwd_27.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_28 = pd.read_csv("./sample_over3_28000.csv")
cat_kwd_28 = cat_kwd_28.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_29 = pd.read_csv("./sample_over3_29000.csv")
cat_kwd_29 = cat_kwd_29.drop(['Unnamed: 0', 'start'], axis=1)
cat_kwd_299 = pd.read_csv("./sample_over3_29900.csv")
cat_kwd_299 = cat_kwd_299.drop(['Unnamed: 0', 'start'], axis=1)

In [102]:
act_kwd_over3[2]

'루이비통'

In [103]:
cat_kwd_1.head(3)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,명품/쥬얼리,명품/쥬얼리,명품/쥬얼리,명품/쥬얼리,패션의류/언더웨어,식품,패션의류/언더웨어,식품,가구/침구/인테리어,검색이 되지 않는다,...,가구/침구/인테리어,식품,주방/생활/건강,식품,식품,식품,패션의류/언더웨어,패션의류/언더웨어,가구/침구/인테리어,가전/디지털
1,수입명품/해외배송,수입명품/해외배송,수입명품/해외배송,수입명품/해외배송,언더웨어,즉석/가공식품,여성트랜드의류,건강식품/홍삼,홈데코/인테리어,NaN,...,홈데코/인테리어,즉석/가공식품,주방,즉석/가공식품,신선식품,즉석/가공식품,여성브랜드의류,여성브랜드의류,홈데코/인테리어,TV/냉장고/세탁기/계절가...
2,스포츠/레저,스포츠/레저,주방/생활/건강,시계/보석/쥬얼리,여성브랜드의류,음료/커피/간식,여성브랜드의류,NaN,침구/커튼/카페트,NaN,...,전문매장,신선식품,문구/악기/도서/취미,여행/렌탈/금융,즉석/가공식품,음료/커피/간식,여성트랜드의류,여성트랜드의류,침실가구,카메라/캠코더/드론


In [104]:
cat_kwd_1_list = list(cat_kwd_1.iloc[0])
cat_kwd_2_list = list(cat_kwd_2.iloc[0])
cat_kwd_3_list = list(cat_kwd_3.iloc[0])
cat_kwd_4_list = list(cat_kwd_4.iloc[0])
cat_kwd_5_list = list(cat_kwd_5.iloc[0])
cat_kwd_6_list = list(cat_kwd_6.iloc[0])
cat_kwd_7_list = list(cat_kwd_7.iloc[0])
cat_kwd_8_list = list(cat_kwd_8.iloc[0])
cat_kwd_9_list = list(cat_kwd_9.iloc[0])
cat_kwd_10_list = list(cat_kwd_10.iloc[0])
cat_kwd_11_list = list(cat_kwd_11.iloc[0])
cat_kwd_12_list = list(cat_kwd_12.iloc[0])
cat_kwd_13_list = list(cat_kwd_13.iloc[0])
cat_kwd_14_list = list(cat_kwd_14.iloc[0])
cat_kwd_15_list = list(cat_kwd_15.iloc[0])
cat_kwd_16_list = list(cat_kwd_16.iloc[0])
cat_kwd_17_list = list(cat_kwd_17.iloc[0])
cat_kwd_18_list = list(cat_kwd_18.iloc[0])
cat_kwd_19_list = list(cat_kwd_19.iloc[0])
cat_kwd_20_list = list(cat_kwd_20.iloc[0])
cat_kwd_21_list = list(cat_kwd_21.iloc[0])
cat_kwd_22_list = list(cat_kwd_22.iloc[0])
cat_kwd_23_list = list(cat_kwd_23.iloc[0])
cat_kwd_24_list = list(cat_kwd_24.iloc[0])
cat_kwd_25_list = list(cat_kwd_25.iloc[0])
cat_kwd_26_list = list(cat_kwd_26.iloc[0])
cat_kwd_27_list = list(cat_kwd_27.iloc[0])
cat_kwd_28_list = list(cat_kwd_28.iloc[0])
cat_kwd_29_list = list(cat_kwd_29.iloc[0])
cat_kwd_299_list = list(cat_kwd_299.iloc[0])

cat_kwd_list = cat_kwd_1_list + cat_kwd_2_list + cat_kwd_3_list + cat_kwd_4_list + cat_kwd_5_list \
               + cat_kwd_6_list + cat_kwd_7_list + cat_kwd_8_list + cat_kwd_9_list + cat_kwd_10_list \
               + cat_kwd_11_list + cat_kwd_12_list + cat_kwd_13_list + cat_kwd_14_list + cat_kwd_15_list \
               + cat_kwd_16_list + cat_kwd_17_list + cat_kwd_18_list + cat_kwd_19_list + cat_kwd_20_list \
               + cat_kwd_21_list + cat_kwd_22_list + cat_kwd_23_list + cat_kwd_24_list + cat_kwd_25_list \
               + cat_kwd_26_list + cat_kwd_27_list + cat_kwd_28_list + cat_kwd_29_list + cat_kwd_299_list

In [105]:
len(cat_kwd_list)

29900

In [106]:
cat_kwd_list

['명품/쥬얼리',
 '명품/쥬얼리',
 '명품/쥬얼리',
 '명품/쥬얼리',
 '패션의류/언더웨어',
 '식품',
 '패션의류/언더웨어',
 '식품',
 '가구/침구/인테리어',
 '검색이 되지 않는다',
 '패션의류/언더웨어',
 '식품',
 '식품',
 '식품',
 '가구/침구/인테리어',
 '식품',
 '식품',
 '검색이 되지 않는다',
 '식품',
 '식품',
 '식품',
 '식품',
 '식품',
 'U아동관',
 '식품',
 '반려동물',
 '식품',
 '명품/쥬얼리',
 '식품',
 '식품',
 '화장품/미용',
 '검색이 되지 않는다',
 '명품/쥬얼리',
 '명품/쥬얼리',
 '명품/쥬얼리',
 '잡화/슈즈',
 '명품/쥬얼리',
 '잡화/슈즈',
 '명품/쥬얼리',
 '잡화/슈즈',
 '스포츠/레저',
 '식품',
 '가구/침구/인테리어',
 '주방/생활/건강',
 '패션의류/언더웨어',
 '패션의류/언더웨어',
 '식품',
 '주방/생활/건강',
 '잡화/슈즈',
 '잡화/슈즈',
 '잡화/슈즈',
 '주방/생활/건강',
 '가전/디지털',
 '가전/디지털',
 '패션의류/언더웨어',
 '화장품/미용',
 '가전/디지털',
 '식품',
 '잡화/슈즈',
 '스포츠/레저',
 '주방/생활/건강',
 '패션의류/언더웨어',
 '주방/생활/건강',
 '검색이 되지 않는다',
 '검색이 되지 않는다',
 '검색이 되지 않는다',
 '식품',
 '스포츠/레저',
 '화장품/미용',
 '검색이 되지 않는다',
 'U아동관',
 '가구/침구/인테리어',
 '식품',
 '식품',
 '식품',
 '명품/쥬얼리',
 '패션의류/언더웨어',
 '주방/생활/건강',
 '주방/생활/건강',
 '식품',
 '주방/생활/건강',
 '여행/렌탈/금융',
 '패션의류/언더웨어',
 '패션의류/언더웨어',
 '가전/디지털',
 '잡화/슈즈',
 '잡화/슈즈',
 '화장품/미용',
 '화장품/미용',
 '화장품/미용',
 '잡화/슈즈',
 '명품/쥬얼리',
 '명품/쥬얼리

In [107]:
cat_1kwd_1_list = list(cat_kwd_1.iloc[1])
cat_1kwd_2_list = list(cat_kwd_2.iloc[1])
cat_1kwd_3_list = list(cat_kwd_3.iloc[1])
cat_1kwd_4_list = list(cat_kwd_4.iloc[1])
cat_1kwd_5_list = list(cat_kwd_5.iloc[1])
cat_1kwd_6_list = list(cat_kwd_6.iloc[1])
cat_1kwd_7_list = list(cat_kwd_7.iloc[1])
cat_1kwd_8_list = list(cat_kwd_8.iloc[1])
cat_1kwd_9_list = list(cat_kwd_9.iloc[1])
cat_1kwd_10_list = list(cat_kwd_10.iloc[1])
cat_1kwd_11_list = list(cat_kwd_11.iloc[1])
cat_1kwd_12_list = list(cat_kwd_12.iloc[1])
cat_1kwd_13_list = list(cat_kwd_13.iloc[1])
cat_1kwd_14_list = list(cat_kwd_14.iloc[1])
cat_1kwd_15_list = list(cat_kwd_15.iloc[1])
cat_1kwd_16_list = list(cat_kwd_16.iloc[1])
cat_1kwd_17_list = list(cat_kwd_17.iloc[1])
cat_1kwd_18_list = list(cat_kwd_18.iloc[1])
cat_1kwd_19_list = list(cat_kwd_19.iloc[1])
cat_1kwd_20_list = list(cat_kwd_20.iloc[1])
cat_1kwd_21_list = list(cat_kwd_21.iloc[1])
cat_1kwd_22_list = list(cat_kwd_22.iloc[1])
cat_1kwd_23_list = list(cat_kwd_23.iloc[1])
cat_1kwd_24_list = list(cat_kwd_24.iloc[1])
cat_1kwd_25_list = list(cat_kwd_25.iloc[1])
cat_1kwd_26_list = list(cat_kwd_26.iloc[1])
cat_1kwd_27_list = list(cat_kwd_27.iloc[1])
cat_1kwd_28_list = list(cat_kwd_28.iloc[1])
cat_1kwd_29_list = list(cat_kwd_29.iloc[1])
cat_1kwd_299_list = list(cat_kwd_299.iloc[1])

cat_1kwd_list = cat_1kwd_1_list + cat_1kwd_2_list + cat_1kwd_3_list + cat_1kwd_4_list + cat_1kwd_5_list \
               + cat_1kwd_6_list + cat_1kwd_7_list + cat_1kwd_8_list + cat_1kwd_9_list + cat_1kwd_10_list \
               + cat_1kwd_11_list + cat_1kwd_12_list + cat_1kwd_13_list + cat_1kwd_14_list + cat_1kwd_15_list \
               + cat_1kwd_16_list + cat_1kwd_17_list + cat_1kwd_18_list + cat_1kwd_19_list + cat_1kwd_20_list \
               + cat_1kwd_21_list + cat_1kwd_22_list + cat_1kwd_23_list + cat_1kwd_24_list + cat_1kwd_25_list \
               + cat_1kwd_26_list + cat_1kwd_27_list + cat_1kwd_28_list + cat_1kwd_29_list + cat_1kwd_299_list

In [108]:
len(cat_1kwd_list)

29900

In [109]:
cat_1kwd_list

['수입명품/해외배송',
 '수입명품/해외배송',
 '수입명품/해외배송',
 '수입명품/해외배송',
 '언더웨어',
 '즉석/가공식품',
 '여성트랜드의류',
 '건강식품/홍삼',
 '홈데코/인테리어',
 nan,
 '여성트랜드의류',
 '신선식품',
 '신선식품',
 '신선식품',
 '홈데코/인테리어',
 '신선식품',
 '즉석/가공식품',
 nan,
 '음료/커피/간식',
 '즉석/가공식품',
 '즉석/가공식품',
 '건강식품/홍삼',
 '즉석/가공식품',
 '임신/출산/기저귀/분유',
 '신선식품',
 '반려동물',
 '음료/커피/간식',
 '수입명품/해외배송',
 '즉석/가공식품',
 '신선식품',
 '화장품/향수',
 nan,
 '수입명품/해외배송',
 '수입명품/해외배송',
 '수입명품/해외배송',
 '패션잡화',
 '수입명품/해외배송',
 '패션잡화',
 '수입명품/해외배송',
 '패션잡화',
 '스포츠패션/슈즈',
 '음료/커피/간식',
 '홈데코/인테리어',
 '세제/욕실/위생',
 '여성브랜드의류',
 '여성브랜드의류',
 '즉석/가공식품',
 '세제/욕실/위생',
 '패션잡화',
 '패션잡화',
 '패션잡화',
 '세제/욕실/위생',
 'TV/냉장고/세탁기/계절가...',
 'TV/냉장고/세탁기/계절가...',
 '남성캐주얼',
 '화장품/향수',
 'TV/냉장고/세탁기/계절가...',
 '신선식품',
 '패션잡화',
 '등산/캠핑/자동차',
 '수납/청소/생활잡화',
 '진/유니섹스',
 '주방',
 nan,
 nan,
 nan,
 '즉석/가공식품',
 '스포츠패션/슈즈',
 '헤어/바디/미용기기',
 nan,
 '유아동도서/완구/교구',
 '홈데코/인테리어',
 '신선식품',
 '건강식품/홍삼',
 '건강식품/홍삼',
 '수입명품/해외배송',
 '여성브랜드의류',
 '주방',
 '주방',
 '음료/커피/간식',
 '수납/청소/생활잡화',
 '상품권/이용권',
 '여성트랜드의류',
 '진/유니섹스',
 '컴퓨터/주변기기/게임',
 '패션슈

In [ ]:
-------------------------------------------------------------------------------------------------------------------

* 중복 제거한 카테고리 키워드 갯수 확인

In [112]:
cat_kwd_set = set(cat_kwd_list)

In [113]:
len(cat_kwd_set)

20

In [114]:
cat_kwd_set

{'TV쇼핑',
 'U아동관',
 'e카탈로그',
 '가구/침구/인테리어',
 '가전/디지털',
 '검색이 되지 않는다',
 '롯데백화점',
 '롯데아울렛',
 '명품/쥬얼리',
 '반려동물',
 '스포츠/레저',
 '식품',
 '아예 검색 없음',
 '여행/렌탈/금융',
 '영플라자',
 '잡화/슈즈',
 '전문매장',
 '주방/생활/건강',
 '패션의류/언더웨어',
 '화장품/미용'}

In [115]:
cat_1kwd_set = set(cat_1kwd_list)
len(cat_1kwd_set)

72

In [116]:
cat_1kwd_set

{'1300K',
 'TV/냉장고/세탁기/계절가...',
 'TV쇼핑',
 'e카탈로그',
 nan,
 '거실가구',
 '건강/다이어트/실버',
 '건강식품/홍삼',
 '골프패션/클럽',
 '남성의류',
 '남성정장/셔츠',
 '남성캐주얼',
 '다이어트식품',
 '등산/캠핑/자동차',
 '디자이너슈즈',
 '렌탈/배달/케어',
 '리모델링/시공가구',
 '명품화장품',
 '문구/악기/도서/취미',
 '반려동물',
 '상품권/이용권',
 '생활/가전',
 '세제/욕실/위생',
 '수납/청소/생활잡화',
 '수입명품/해외배송',
 '스포츠/아웃도어',
 '스포츠패션/슈즈',
 '시계/보석/쥬얼리',
 '식품',
 '신선식품',
 '아동의류/잡화',
 '아웃도어/골프/레저',
 '언더웨어',
 '여성브랜드의류',
 '여성의류',
 '여성정장/모피',
 '여성캐주얼',
 '여성트랜드의류',
 '여행/숙박/항공권',
 '영캐주얼',
 '유모차/카시트/가구/매트',
 '유아동도서/완구/교구',
 '유아동샵',
 '유아동의류/잡화',
 '유아복/출산용품',
 '음료/커피/간식',
 '임신/출산/기저귀/분유',
 '장터_라운지',
 '주방',
 '주방/생활/이미용가전',
 '즉석/가공식품',
 '진/유니섹스',
 '진유니섹스',
 '친환경SHOP',
 '침구/커튼/카페트',
 '침실가구',
 '카메라/캠코더/드론',
 '컴퓨터/주변기기/게임',
 '키친갤러리',
 '텐바이텐',
 '패션슈즈',
 '패션슈즈/악세서리',
 '패션의류/언더웨어',
 '패션잡화',
 '하이마트',
 '하프클럽',
 '학생/아동/주방수납',
 '헤어/바디/미용기기',
 '헬스/자전거/수영/스키',
 '홈데코/인테리어',
 '화장품/향수',
 '휴대폰/음향/차량용기기'}

 * 3번 이상 검색된 키워드와 그 키워드 검색에 따른 롯데홈쇼핑 카테고리를 연결한 table 데이터 선언
     * sech_kwd_over_3 : 3번 이상 검색된 키워드
     * sech_kwd_cat : 3번 이상 검색된 키워드를 롯데홈쇼핑몰에서 검색하여 나온 카테고리 키워드

In [117]:
sech_kwd_cat_dict = {'sech_kwd_over_3':act_kwd_over3, 'sech_kwd_cat_0':cat_kwd_list, 'sech_kwd_cat_1':cat_1kwd_list}
sech_kwd_cat_table = pd.DataFrame(sech_kwd_cat_dict)

In [118]:
sech_kwd_cat_table

,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1
0,펜디가방,명품/쥬얼리,수입명품/해외배송
1,페레가모가방,명품/쥬얼리,수입명품/해외배송
2,루이비통,명품/쥬얼리,수입명품/해외배송
3,구찌,명품/쥬얼리,수입명품/해외배송
4,속옷세트,패션의류/언더웨어,언더웨어
...,...,...,...
29895,쉬폰밴딩팬츠,패션의류/언더웨어,여성브랜드의류
29896,anf 유기농 양고기,반려동물,반려동물
29897,드립 여과지,주방/생활/건강,주방
29898,옥수수수프,식품,음료/커피/간식


 * Table 파일 저장

In [119]:
sech_kwd_cat_table.to_csv("sech_kwd01_cat.csv", mode='w')

 * cat1과 대분류를 매칭시킨 table 불러오기

In [123]:
cat1_to_clac_nm1 = pd.read_csv("kwd_to_clac_nm1.csv", header=0)

In [124]:
cat1_to_clac_nm1

,sech_kwd_cat_1,clac_nm1
0,즉석/가공식품,Cooked Foods
1,여성트랜드의류,Women's Clothing
2,음료/커피/간식,Snack Foods
3,스포츠패션/슈즈,Sport Fashion
4,신선식품,Chilled Foods
...,...,...
66,TV쇼핑,NaN
67,여성의류,Women's Clothing
68,디자이너슈즈,Fashion Accessories
69,장터_라운지,NaN


In [126]:
sech_kwd_cat_table = pd.merge(sech_kwd_cat_table, cat1_to_clac_nm1, on = 'sech_kwd_cat_1', how = 'left')

In [132]:
sech_kwd_cat_table.rename(columns = {'clac_nm1' : 'sech_clac_nm1'}, inplace = True)

In [133]:
sech_kwd_cat_table

,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1
0,펜디가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
1,페레가모가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
2,루이비통,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
3,구찌,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
4,속옷세트,패션의류/언더웨어,언더웨어,Underwear / Socks and Hosiery / Homewear
...,...,...,...,...
29895,쉬폰밴딩팬츠,패션의류/언더웨어,여성브랜드의류,Women's Clothing
29896,anf 유기농 양고기,반려동물,반려동물,Gardening / Pets
29897,드립 여과지,주방/생활/건강,주방,Kitchenware
29898,옥수수수프,식품,음료/커피/간식,Snack Foods


In [141]:
merge_table.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,de_tm,buy_am,buy_ct,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2,act_kwd_over3
0,72429,1,2,2,A03,20190919,22:09,842130,NaN,NaN,...,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN
1,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,...,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN
2,72428,5,1,0,A01,20190918,06:37,12568,NaN,라인에디션블라우스,...,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN
3,72428,4,1,0,A01,20190914,23:42,243555,NaN,펜디가방,...,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,펜디가방
4,72428,3,3,0,A01,20190914,21:50,839771,NaN,페레가모가방,...,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,페레가모가방


 ## 입력 데이터 병합

In [142]:
merge_table = pd.merge(merge_table, sech_kwd_cat_table, left_on = 'act_kwd_over3', right_on ='sech_kwd_over_3', how='left')

In [143]:
merge_table

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2,act_kwd_over3,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1
0,72429,1,2,2,A03,20190919,22:09,842130,NaN,NaN,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72428,5,1,0,A01,20190918,06:37,12568,NaN,라인에디션블라우스,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72428,4,1,0,A01,20190914,23:42,243555,NaN,펜디가방,...,unknown,NaN,NaN,NaN,NaN,펜디가방,펜디가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
4,72428,3,3,0,A01,20190914,21:50,839771,NaN,페레가모가방,...,unknown,NaN,NaN,NaN,NaN,페레가모가방,페레가모가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196358,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196359,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,...,unknown,NaN,NaN,NaN,NaN,과일바구니,과일바구니,여행/렌탈/금융,렌탈/배달/케어,Living / Rental Services
3196360,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 * 중복 열 제거
 

In [144]:
merge_table.drop('act_kwd_over3', axis=1, inplace=True)

In [145]:
merge_table

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1
0,72429,1,2,2,A03,20190919,22:09,842130,NaN,NaN,...,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,...,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72428,5,1,0,A01,20190918,06:37,12568,NaN,라인에디션블라우스,...,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72428,4,1,0,A01,20190914,23:42,243555,NaN,펜디가방,...,unknown,unknown,NaN,NaN,NaN,NaN,펜디가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
4,72428,3,3,0,A01,20190914,21:50,839771,NaN,페레가모가방,...,unknown,unknown,NaN,NaN,NaN,NaN,페레가모가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,...,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196358,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,...,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196359,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,...,unknown,unknown,NaN,NaN,NaN,NaN,과일바구니,여행/렌탈/금융,렌탈/배달/케어,Living / Rental Services
3196360,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,...,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
merge_table.to_csv("merge_sech_clac_nm1.csv")

### 대분류(clac_nm1) 관련 EDA

 * 온라인 행동 정보를 활용하여 대분류 설정 : 1572140개

In [161]:
len(merge_table[merge_table['sech_clac_nm1'].isna() == False])

1572140

 * 실제 구매이력으로 부터 대분류 정보가 있는 갯수 : 5733개

In [160]:
len(merge_table[merge_table['clac_nm1'].isna() == False])

5733

 * 실제 구매한 상품의 대분류와 , 검색어를 통해 확인한 대분류의 차이 : 802개 다름

In [157]:
merge_table[['clac_nm1', 'sech_clac_nm1']][(merge_table['clac_nm1'].isna() == False) & (merge_table['sech_clac_nm1'].isna() == False) & (merge_table['clac_nm1'] != merge_table['sech_clac_nm1'])]

,clac_nm1,sech_clac_nm1
1712,Beverages,Snack Foods
1793,Cell Phones / Accessories,Cosmetics / Beauty Care
9110,Grains,Cooked Foods
9166,Fruits,Chilled Foods
9176,Grains,Chilled Foods
...,...,...
3173753,Refrigerators and Washing Machines,Heating / Cooling Electronics
3187821,Kids' Clothing,Birth Supplies / Baby Products
3193697,Grains,Chilled Foods
3196194,Beverages,Books / Records / Instruments
